<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Datafy_Workshop/LlamaIndex/llamaindex_ReAct_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building and Testing a ReAct Agent with LlamaIndex: From Simple Tools to QueryEngine Integration
## **Introduction:**
This notebook demonstrates how to create and utilize a ReAct (Reasoning and Acting) agent using LlamaIndex. The ReAct agent is designed to interact with various tools, starting with simple arithmetic operations and progressing to more complex tasks like querying financial data from Uber and Lyft's 10-K filings. The notebook provides a step-by-step guide to setting up the agent, integrating tools, and performing queries, making it a valuable resource for understanding how to build intelligent agents for task automation and data analysis.

## **Goal:**
The goal of this notebook is to:

1. Introduce the concept of a ReAct agent and its ability to reason and act using tools.
2. Demonstrate how to create a ReAct agent with simple tools (e.g., arithmetic operations).
3. Extend the agent's capabilities by integrating QueryEngine tools for querying financial data from Uber and Lyft's 10-K filings.
4. Providing a hands-on example of how to use the agent to answer complex questions by combining reasoning and tool usage.

## Installation

In [32]:
%pip install -qU llama-index llama-index-llms-gemini llama-index-embeddings-huggingface

## Setup API Keys

In [33]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
from google.colab import userdata

os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

from IPython.display import display, HTML

## Set LLM and Embedding model
We will use meta latest released `llama-3.3-70b-versatile` LLM.

In [37]:
# from llama_index.llms.groq import Groq
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [36]:
llm = Gemini(model="models/gemini-1.5-flash")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [38]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

In [39]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [40]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)

## Create ReAct Agent
Create agent over tools and test out queries

In [41]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [42]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

> Running step 77f5cbc1-04e6-4c9d-86a5-0db3755f17b9. Step input: What is 20+(2*4)? Calculate step by step 
Thought: The current language of the user is: English. I need to use tools to calculate 20+(2*4) step by step.  First I will multiply 2 and 4, then add the result to 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step e3da7de3-3565-41fa-aa25-2da715107dca. Step input: None
Thought: I have the result of the multiplication (8). Now I need to add this to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step ea11ab26-d3a1-4b52-b751-069a114ec73a. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 28


In [43]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

## Visit Prompts
You can check prompts that the agent used to select the tools.

In [44]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may

## ReAct Agent over `QueryEngine` Tools

In [45]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

## Download data
We will define ReAct agent over tools created on QueryEngines with Uber and Lyft 10K SEC Filings.

In [46]:
!mkdir -p 'data/10k/'
!wget 'https://github.com/run-llama/llama_index/raw/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://github.com/run-llama/llama_index/raw/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2025-01-22 13:12:11--  https://github.com/run-llama/llama_index/raw/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf [following]
--2025-01-22 13:12:12--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.007s  

2025-01-22 13:12

## Load Data

In [47]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

lyft_docs = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["./data/10k/uber_2021.pdf"]).load_data()

## Build Index

In [48]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

## Create QueryEngines

In [49]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

### Create QueryEngine Tools

In [50]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Querying with ReAct Agent

In [51]:
response = agent.chat("What is Lyft's core business model, and how does it generate revenue?")

> Running step 65b1e4d2-82d0-4b56-8cbb-8187e94f2d50. Step input: What is Lyft's core business model, and how does it generate revenue?
Thought: I need to use external knowledge to answer this question about Lyft's business model and revenue generation.  I don't have access to tools to directly research this.
Answer: Lyft's core business model is a ride-sharing platform connecting passengers with drivers through a mobile app.  They generate revenue primarily through commissions charged to drivers on each fare.  This commission is a percentage of the fare paid by the passenger.  Lyft may also generate additional revenue through advertising and other ancillary services offered through the app.


In [52]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [53]:
response = agent.chat("Compare and contrast the revenue growth of Uber and Lyft in 2021, then" "give an analysis")

> Running step eeb7b9c5-d627-4b23-b763-8468b498bb3c. Step input: Compare and contrast the revenue growth of Uber and Lyft in 2021, thengive an analysis
Thought: (Implicit) I can answer without any more tools!
Answer: I do not have access to real-time information, including live financial data for companies like Uber and Lyft.  To compare and contrast their revenue growth in 2021, you would need to consult their respective financial reports (10-K filings) for that year.  These reports are usually publicly available on the companies' investor relations websites and through the SEC's EDGAR database.

Once you obtain the necessary data (likely net revenue or gross booking figures for both companies), you can compare the growth rates using the following formula:

`(Revenue in 2021 - Revenue in 2020) / Revenue in 2020 * 100%`

This will give you the percentage growth for each company.  You can then compare these percentages to see which company experienced higher revenue growth in 2021.

**A

In [54]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))